In [0]:
!pip install yfinance
!pip install azure-eventhub

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 98.2 MB/s eta 0:00:00
  Created wheel for multitasking: filename=multitasking-0.0.12-py3-none-any.whl size=15548 sha256=210a7aa580da037af8cb85701edc0f3d42ab616e80ee8391e711c78ca80788ab
  Stored in directory: /root/.cache/pip/wheels/cc/bd/6f/664d62c99327abeef7d86489e6631cbf45b56fbf7ef1d6ef00
Successfully built multitasking
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
import sys
import time
import json
from datetime import datetime, timezone
from zoneinfo import ZoneInfo
from typing import Dict, List, Optional

import yfinance as yf
from azure.eventhub import EventHubProducerClient, EventData
from pyspark.sql.functions import year, month, dayofmonth, hour

In [0]:
# ======================= EVENT HUB =======================
from azure.eventhub import EventHubProducerClient, EventData

connection_str = "Endpoint=sb://mastertfm001eh.servicebus.windows.net/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey=Rd8FIOZwcSLilXZ7jC/Zb6XMIMUgQHGJi+AEhGlvJ9Y="
eventhub_name = "mastertfm001eh-eh"

producer = EventHubProducerClient.from_connection_string(
    conn_str=connection_str,
    eventhub_name=eventhub_name
)

try:
    batch = producer.create_batch()
    batch.add(EventData("ping"))
    producer.send_batch(batch)
    print("✅ Autenticación OK")
except Exception as e:
    print("❌ Fallo autenticación:", e)



✅ Autenticación OK


In [0]:
# ====================================================================
# ========== INGESTA OHLCV 1m DESDE YFINANCE A BRONZE =================
# ====================================================================

import time
from datetime import datetime, timezone, date
from zoneinfo import ZoneInfo
from typing import Dict, List

import yfinance as yf
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, year, month, dayofmonth, hour
)

# ================= CONFIGURACIÓN =================

TZ_DEFAULT = "Europe/Madrid"
INTERVAL = "1m"
PERIOD = "1d"
POLLING_INTERVAL_SEC = 60

bronze_base_path = (
    "abfss://datos@mastertfm001sta.dfs.core.windows.net/bronze/activos"
)

# 🔵 Spark con soporte Delta
spark = (
    SparkSession.builder
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .getOrCreate()
)
local_tz = ZoneInfo(TZ_DEFAULT)

# ================= ACTIVOS =================

ASSET_GROUPS: Dict[str, List[Dict[str, str]]] = {
    "Acciones(S&P500)": [
        {"symbol": "TSLA", "name": "Tesla"},
        {"symbol": "NVDA", "name": "Nvidia"},
        {"symbol": "AMD", "name": "AMD"},
        {"symbol": "COIN", "name": "Coinbase"},
        {"symbol": "PLTR", "name": "Palantir"},
        {"symbol": "RIVN", "name": "Rivian"},
        {"symbol": "SHOP", "name": "Shopify"},
        {"symbol": "LCID", "name": "Lucid Motors"},
        {"symbol": "ZM", "name": "Zoom"},
        {"symbol": "SPCE", "name": "Virgin Galactic"},
        {"symbol": "KO", "name": "Coca Cola Company"},
        {"symbol": "PG", "name": "Procter & Gamble"},
        {"symbol": "JNJ", "name": "Johnson & Johnson"},
        {"symbol": "PEP", "name": "PepsiCo"},
        {"symbol": "WMT", "name": "Walmart"},
        {"symbol": "MCD", "name": "McDonalds"},
        {"symbol": "VZ", "name": "Verizon"},
        {"symbol": "DUK", "name": "Duke Energy"},
        {"symbol": "UL", "name": "Unilever"},
        {"symbol": "V", "name": "VISA"},
    ],
    "Fondos(ETFs)": [
        {"symbol": "SPY", "name": "SPDR S&P 500 ETF Trust"},
        {"symbol": "QQQ", "name": "Invesco QQQ Trust"},
        {"symbol": "EEM", "name": "iShares MSCI Emerging Markets ETF"},
        {"symbol": "VGK", "name": "Vanguard FTSE Europe ETF"},
        {"symbol": "AGG", "name": "iShares Core U.S. Aggregate Bond ETF"},
        {"symbol": "VNQ", "name": "Vanguard Real Estate ETF"},
        {"symbol": "ARKK", "name": "ARK Innovation ETF"},
        {"symbol": "VUG", "name": "Vanguard Growth ETF"},
        {"symbol": "SCHD", "name": "Schwab US Dividend Equity ETF"},
        {"symbol": "SOXX", "name": "iShares Semiconductor ETF"},
    ],
    "Forex(TCambio)": [
        {"symbol": "EURUSD=X", "name": "Euro / Dólar"},
        {"symbol": "USDJPY=X", "name": "Dólar / Yen"},
        {"symbol": "GBPUSD=X", "name": "Libra / Dólar"},
        {"symbol": "USDCHF=X", "name": "Dólar / Franco Suizo"},
        {"symbol": "AUDUSD=X", "name": "Dólar Australiano / Dólar"},
        {"symbol": "USDCAD=X", "name": "Dólar / Dólar Canadiense"},
        {"symbol": "NZDUSD=X", "name": "Dólar Nueva Zelanda / Dólar"},
        {"symbol": "EURGBP=X", "name": "Euro / Libra"},
        {"symbol": "EURJPY=X", "name": "Euro / Yen"},
        {"symbol": "GBPJPY=X", "name": "Libra / Yen"},
    ],
    "Cripto(vsUSD)": [
        {"symbol": "BTC-USD", "name": "Bitcoin"},
        {"symbol": "ETH-USD", "name": "Ethereum"},
        {"symbol": "BNB-USD", "name": "Binance Coin"},
        {"symbol": "XRP-USD", "name": "Ripple"},
        {"symbol": "SOL-USD", "name": "Solana"},
        {"symbol": "TRX-USD", "name": "TRON"},
        {"symbol": "DOGE-USD", "name": "Dogecoin"},
        {"symbol": "ADA-USD", "name": "Cardano"},
	    {"symbol": "AVAX-USD", "name": "Avalanche"},
        {"symbol": "LTC-USD", "name": "Litecoin"},     
    ],
    "Commodities(ETF)": [
        {"symbol": "GLD", "name": "Oro"},
        {"symbol": "SLV", "name": "Plata"},
        {"symbol": "PPLT", "name": "Platino"},
        {"symbol": "PALL", "name": "Paladio"},
        {"symbol": "USO", "name": "Petróleo"},
        {"symbol": "UNG", "name": "Gas Natural"},
        {"symbol": "CORN", "name": "Maiz"},
        {"symbol": "SOYB", "name": "Soja"},
        {"symbol": "WEAT", "name": "Trigo"},
        {"symbol": "CANE", "name": "Azucar"},
    ],
}

ALL_SYMBOLS = [
    item["symbol"]
    for group in ASSET_GROUPS.values()
    for item in group
]

# ================= UTIL =================

def log(msg: str):
    print(f"[INFO] {msg}", flush=True)

# ================= YFINANCE =================

def fetch_ohlcv(symbol: str):
    """
    Descarga velas OHLCV 1m cerradas
    """
    df = yf.download(
        tickers=symbol,
        interval=INTERVAL,
        period=PERIOD,
        progress=False
    )

    if df.empty:
        return None

    df = df.reset_index()
    df.columns = [c[0] if isinstance(c, tuple) else c for c in df.columns]

    # 🔹 Forzar Datetime tz-aware UTC
    df["Datetime"] = df["Datetime"].dt.tz_localize(None).dt.tz_localize("UTC")

    # 🔒 CAST CRÍTICO
    df["Volume"] = df["Volume"].astype("float64")

    clean_symbol = symbol.replace("=X", "")

    df["symbol"] = clean_symbol
    df["timezone"] = TZ_DEFAULT
    df["source"] = "yfinance"

    return df

# ================= BRONZE =================

def write_ohlcv_to_bronze(df):
    """
    Normaliza columnas y escribe en Bronze particionado
    """
    df = (
        df
        .withColumnRenamed("Datetime", "timestamp")
        .withColumnRenamed("Open", "open")
        .withColumnRenamed("High", "high")
        .withColumnRenamed("Low", "low")
        .withColumnRenamed("Close", "close")
        .withColumnRenamed("Volume", "volume")
        .withColumn("year", year("timestamp"))
        .withColumn("month", month("timestamp"))
        .withColumn("day", dayofmonth("timestamp"))
        .select(
            "timestamp",
            "symbol",
            "open",
            "high",
            "low",
            "close",
            "volume",
            "timezone",
            "source",
            "year",
            "month",
            "day",
        )
    )

    # 🔵 ESCRITURA EN DELTA
    (
        df.write
        .format("delta")
        .mode("append")
        .partitionBy("year", "month", "day")
        .save(bronze_base_path)
    )

# ================= LOOP PRINCIPAL =================

def run():
    log(f"Zona horaria: {TZ_DEFAULT}")
    log(f"Tickers: {len(ALL_SYMBOLS)}")
    log("Iniciando ingesta OHLCV 1m (solo nuevas y solo hoy)...")

    # 🟢 Guardar último timestamp por ticker
    last_timestamp_per_symbol = {symbol: None for symbol in ALL_SYMBOLS}

    while True:
        start = time.time()
        today_utc = datetime.now(timezone.utc).date()

        for symbol in ALL_SYMBOLS:
            try:
                df = fetch_ohlcv(symbol)
                if df is None or df.empty:
                    log(f"{symbol}: no hay datos para hoy")
                    continue

                # 🔹 Filtrar solo nuevas velas
                last_ts = last_timestamp_per_symbol[symbol]
                if last_ts is not None:
                    df = df[df["Datetime"] > last_ts]

                # 🔹 Filtrar solo hoy
                df = df[df["Datetime"].dt.date == today_utc]

                if df.empty:
                    log(f"{symbol}: no hay datos nuevos para hoy")
                    continue

                # 🔹 Crear Spark DF y escribir en Bronze
                spark_df = spark.createDataFrame(df)
                write_ohlcv_to_bronze(spark_df)

                # 🔹 Actualizar last_timestamp por ticker
                last_timestamp_per_symbol[symbol] = df["Datetime"].max()

                log(f"{symbol}: OHLCV escrito | Rango de datos: "
                    f"{df['Datetime'].min()} -> {df['Datetime'].max()}")

            except Exception as e:
                log(f"{symbol}: error -> {e}")

        elapsed = time.time() - start
        sleep_time = POLLING_INTERVAL_SEC - elapsed
        if sleep_time > 0:
            time.sleep(sleep_time)

# ================= MAIN =================

if __name__ == "__main__":
    run()



[INFO] Zona horaria: Europe/Madrid
[INFO] Tickers: 60
[INFO] Iniciando ingesta OHLCV 1m (solo nuevas y solo hoy)...
[INFO] TSLA: OHLCV escrito | Rango de datos: 2026-01-30 14:30:00+00:00 -> 2026-01-30 17:37:00+00:00
[INFO] NVDA: OHLCV escrito | Rango de datos: 2026-01-30 14:30:00+00:00 -> 2026-01-30 17:37:00+00:00
[INFO] AMD: OHLCV escrito | Rango de datos: 2026-01-30 14:30:00+00:00 -> 2026-01-30 17:37:00+00:00
[INFO] COIN: OHLCV escrito | Rango de datos: 2026-01-30 14:30:00+00:00 -> 2026-01-30 17:37:00+00:00
[INFO] PLTR: OHLCV escrito | Rango de datos: 2026-01-30 14:30:00+00:00 -> 2026-01-30 17:37:00+00:00
[INFO] RIVN: OHLCV escrito | Rango de datos: 2026-01-30 14:30:00+00:00 -> 2026-01-30 17:37:00+00:00
[INFO] SHOP: OHLCV escrito | Rango de datos: 2026-01-30 14:30:00+00:00 -> 2026-01-30 17:37:00+00:00
[INFO] LCID: OHLCV escrito | Rango de datos: 2026-01-30 14:30:00+00:00 -> 2026-01-30 17:37:00+00:00
[INFO] ZM: OHLCV escrito | Rango de datos: 2026-01-30 14:30:00+00:00 -> 2026-01-30 17

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:132)
	at scala.Option.getOrElse(Option.scala:201)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:132)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:190)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:201)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

In [0]:
#import yfinance as yf

#df = yf.download("MSFT", interval="1m", period="1d", progress=False)
#print(df.index.max())


com.databricks.backend.common.rpc.CommandSkippedException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:134)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:190)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:201)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:465)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:750)
	at com.data

In [0]:
# bronze_path = "abfss://datos@mastertfm001sta.dfs.core.windows.net/bronze/activos"
# dbutils.fs.rm(bronze_path, recurse=True)


True